In [17]:
%%capture
!pip install deicode
!pip install fancyimpute
import numpy as np
from deicode.matrix_completion import MatrixCompletion

In [18]:
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler
from IPython.utils import io
import collections
import math
import timeit
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
class testresults:
  def __init__(self):
    self.n=0
    self.iter=0
    self.kf=0
    self.k=0
    self.duration=0
    self.min_upsets=0
    self.upsets = {
        "optspace" : 0,
        "nm":0,
        "fancyimpute" : 0 
    }

In [20]:
class PBR2R():
  def __init__(self, Matrix, P_true):
    self.Tournament = Matrix>Matrix.T
    self.True_tournament = P_true>P_true.T
    #No. of vertices
    self.V, _= self.Tournament.shape
    self.Time = 0
    self.scc=[]
    
  def SCCUtil(self,u, low, disc, stackMember, st):
    
    disc[u] = self.Time
    low[u] = self.Time
    self.Time += 1
    stackMember[u] = True
    st.append(u)
    
    x = np.where(self.Tournament[u] == 1)
    for v in x[0]:
      if disc[v] == -1 :
        self.SCCUtil(v, low, disc, stackMember, st)
        low[u] = min(low[u], low[v])
      
      elif stackMember[v] == True:
        low[u] = min(low[u], disc[v])
      
    w = -1
    if low[u] == disc[u]:
      temp = []
      while w != u:
        w = st.pop()
        temp.append(w)
        stackMember[w] = False
      self.scc.append(temp)
        
  def SCC(self):
    disc = [-1] * (self.V)
    low = [-1] * (self.V)
    stackMember = [False] * (self.V)
    st =[]
    
    for i in range(self.V):
      if disc[i] == -1:
        self.SCCUtil(i, low, disc, stackMember, st)

  def Sub_tournament(self, Vertices, val=False):

    if not val:
      Sub_tournament = self.Tournament[np.ix_(np.array(Vertices), np.array(Vertices))]
    else:
      Sub_tournament = self.True_tournament[np.ix_(np.array(Vertices), np.array(Vertices))]
    return Sub_tournament
  
  def flatten(self, lis):
    for item in lis:
      if isinstance(item, collections.Iterable) and not isinstance(item, str):
        for x in self.flatten(item):
          yield x
      else:        
        yield item
  
  def Triangle(self, Tournament):
    
    n,m = Tournament.shape
    
    for u in range(n):
      x = np.where(Tournament[u] == 1)
      for v in x[0]:
        y = np.where(Tournament[v] == 1)
        for w in y[0]:
          if Tournament[w][u]:
            return [u,v,w]
    
    return []

  def Topologicalsort(self, Tournament, Vertices):
    
    n,_ = Tournament.shape
    degree = np.zeros(shape = n, dtype = int)
    
    for u in range(n):
      x= np.where(Tournament[u] == 1)
      for v in x[0]:
        degree[v]+=1
        
    queue=[]
    x= np.where(degree == 0)
    for u in x[0]:
      queue.append(u)
      
    count = 0
    Topologicalorder =[]
    
    while queue:
      u = queue.pop(0)
      Topologicalorder.append(Vertices[u])
      
      x= np.where(Tournament[u] == 1)
      for v in x[0]:
        degree[v]-=1
        if degree[v] == 0:
          queue.append(v)
      count+=1
        
    if count!=n:
      return (False, [])
    else:
      return (True, Topologicalorder)
  
  def find(self, Tournament, Vertices, cycle):

    n, _ = Tournament.shape
    Splus, Sminus, A, B, C = [], [], [], [], []

    # cycle triangle with nodes as "a", "b", "c"
    for i in range(n):
      # if node beats every node in triangle add it to Splus
      if Tournament[i][cycle[0]] and Tournament[i][cycle[1]] and Tournament[i][cycle[2]]:
        Splus.append(Vertices[i])

      # if node is beaten by every node in triangele then add it to Sminus
      elif Tournament[cycle[0]][i] and Tournament[cycle[1]][i] and Tournament[cycle[2]][i]:
        Sminus.append(Vertices[i])

      # if node beats "b" and beaten by "c"
      elif Tournament[i][cycle[1]] and Tournament[cycle[2]][i]:
        A.append(Vertices[i])

      # if node beats "c" and beaten by "a"
      elif Tournament[i][cycle[2]] and Tournament[cycle[0]][i]:
        B.append(Vertices[i])

      # if node beats "a" and beaten by "b"
      elif Tournament[i][cycle[0]] and Tournament[cycle[1]][i]:
        C.append(Vertices[i])

      else:
        continue

    return Splus, Sminus, A, B, C

  def Upsets(self, Tournament, sigma):
    n,=sigma.shape
    i_lower = np.tril_indices(n, -1)
    i_upper = np.triu_indices(n, 1)
    return (Tournament[i_lower]>Tournament[i_upper]).sum()

  def MFAST(self, Tournament, Vertices):
    sigma = np.array(Vertices)
    n, = sigma.shape
    upset = n*n
    sigmaStar = sigma.copy()

    for i in range(n):
      rank_eval = self.Upsets(Tournament, sigma)
      if rank_eval<upset:
        upset = rank_eval
        sigmaStar = sigma
      sigma = np.roll(sigma, 1)
      Tournament = np.insert(Tournament, n, Tournament[0], axis=0)
      Tournament = np.delete(Tournament, 0, 0)
      Tournament = np.insert(Tournament, n, Tournament[:,0], axis=1)
      Tournament = np.delete(Tournament, 0, 1)

    return sigmaStar

  def Rank(self, Tournament, Vertices):
    count = np.sum(Tournament, axis=1)
    index = np.argsort(-count)
    rank = [Vertices[i] for i in index]
    return rank

  def BR2R(self, Vertices):

    if len(Vertices) <=1:
      return Vertices

    Tour = self.Sub_tournament(Vertices)
    n, _ = Tour.shape
    flag, Sort = self.Topologicalsort(Tour, Vertices)

    if flag:
      return Sort
    else:
      cycle = self.Triangle(Tour)
      if len(cycle)==0 or len(Vertices)<=3:
        return self.Rank(Tour, Vertices)

      Splus, Sminus, A, B, C = self.find(Tour, Vertices, cycle)
      sigma = [self.BR2R(A), self.BR2R(B), self.BR2R(C)]
      sigma1 = self.BR2R(Splus)
      sigma2 = self.BR2R(Sminus)
      sigma = list(self.flatten(sigma))
      sigma = self.MFAST(self.Sub_tournament(sigma), sigma)

      return list(self.flatten(sigma1)) + list(sigma) + list(self.flatten(sigma2))

  def BR2R_UPSETS(self, blocksize):

    self.SCC()
    blockupsets=0
    Rank=[]
    n = len(self.scc)
    for component in self.scc:
      rank = self.BR2R(component)
      Rank+=rank
      tour = self.Sub_tournament(rank, val=True)
      blockupsets+=self.Upsets(tour, np.array(rank))
    
    multiblock = 0
    for i in range(n):
      for j in range(i+1, n):
        for k in self.scc[i]:
          for l in self.scc[j]:
            if (self.True_tournament[k][l]>self.True_tournament[l][k]) != (self.Tournament[k][l]>self.Tournament[l][k]):
              multiblock+=1 
    # print(multiblock, blockupsets)
    return blockupsets+multiblock

In [21]:
class SyntheticData():

  def __init__(self, n, Block):
    self.n = n
    self.Block_size = n//Block
    self.rank = 4*Block
    self.num_blocks = Block

  def uniform_real(self, rng, low=0.0, high=1.0, size=1):
    return rng.uniform(low=low, high=high, size=size)

  def uniform_int(self, rng, low, high, size=1):
    return rng.integers(low = low, high = high, size=size)

  def TrueP(self, U, V, S):
    P_true = np.zeros(shape= (self.n, self.n), dtype=float)

    for i in range(self.n):
      for j in range(self.n):
        if i//self.Block_size == j//self.Block_size:
          P_true[i][j]=np.exp(U[i]*V[j])
        else:
          P_true[i][j]=np.exp(S[i])
    P_true = P_true/(P_true + P_true.T)
    # with np.errstate(invalid='ignore', divide='ignore'):
    #   P_true = np.log(P_true/P_true.T)
    return P_true

  def Random_shuffle(self, U, V, S, P, rand):
    n,_ = P.shape
    for i in range(n-1):
      j = self.uniform_int(rand, i, n-1)[0]
      U[[i, j]] = U[[j, i]]
      V[[i, j]] = V[[j, i]]
      S[[i, j]] = S[[j, i]]
      P[:,[i,j]] = P[:, [j, i]]
      P[[i,j],:] = P[[j,i],:]

  def KnownValues(self, known_pairs, rand):
    Known = self.uniform_real(rand, size=(self.n, self.n))
    i_lower = np.tril_indices(self.n, -1)
    i_upper = np.triu_indices(self.n, 1)
    Known[i_upper] = Known[i_upper]<=known_pairs
    Known[i_lower] = Known[i_upper]
    np.fill_diagonal(Known, 1)
    return Known

  def Bernouli(self, rand, p, k):
    R = self.uniform_real(rand, size=k)
    count = (R<p).sum()
    if count==0:
      return 0.0001
    if count==k:
      return 1-0.0001
    return count/k

  def Simulation(self, rand, P_true, k):
    if k>10001:
      return P_true.copy()
    
    P_sim = np.zeros(shape= (self.n, self.n), dtype=float)
    for i in range(self.n):
      for j in range(self.n):
        if i==j:
          P_sim[i][j] = 0.5
        elif i<j:
          P_sim[i][j] = self.Bernouli(rand, P_true[i][j], k)
        else:
          P_sim[i][j] = 1-P_sim[j][i]
    
    with np.errstate(invalid='ignore', divide='ignore'):
      P_sim = np.log(P_sim/P_sim.T)
    return P_sim

  def Generate(self, rng, multiplier=1):
    
    # generate random U, V and Score vectors..
    U = self.uniform_real(rng, size=self.n)
    V = self.uniform_real(rng, size=self.n)
    Score = self.uniform_real(rng, size=self.n)
    U = multiplier*(2*U-1)
    V = multiplier*(2*V-1)
    Score = multiplier*Score
    Score[::-1].sort()
    return U, V, Score

  def mask(self, matrix):
    matrix[matrix!=0]=1
    return matrix

  def Matrix_Completion(self, Matrix, r, Mask, steps=1000, mu=0.01, ro=1, l=0.001, eps=1e-4):
    U = np.random.normal(size=(Matrix.shape[0], r))
    V = np.random.normal(size=(Matrix.shape[1], r))
    Z = U@V.T
    Y = np.zeros(shape=(Matrix.shape))
    I = np.eye(r)
    step = 0
    converged, converged2 = False, False

    while not converged:
      step2 =0
      while not converged2:
        U = (ro*Z +Y) @ V @ np.linalg.inv(ro*V.T @ V + l*I )
        V = (ro*Z +Y).T @ U @ np.linalg.inv(ro*U.T @ U + l*I )
        temp = U @V.T -1/ro * Y
        ZZ = Mask*(1/(2+ro) *(2*Matrix + ro*temp)) + (1-Mask)*temp
        converged2 = np.linalg.norm(Z-ZZ)/np.linalg.norm(Z) <eps or step2>steps
        step2+=1
        Z = ZZ.copy()
      temp = Z - U @ V.T
      Y = Y + ro*temp
      ro = min(ro*mu, 1e20)
      converged = np.linalg.norm(temp)<eps or step>steps
      step+=1
    
    Z = U @ V.T
    # with np.errstate(invalid='ignore', divide='ignore'):
    #   Z = np.log10(abs(Z/Z.T))
    return Z

  def fancyimpute(self, Matrix, steps):
    
    Matrix[Matrix==0]=np.nan
    X_incomplete_normalized = BiScaler(max_iters=steps).fit_transform(Matrix)
    P_completed = SoftImpute(max_iters=steps).fit_transform(X_incomplete_normalized)
    np.fill_diagonal(P_completed, 0)
    # with np.errstate(invalid='ignore', divide='ignore'):
    #   P_completed = np.log10(abs(P_completed/P_completed.T))
    return P_completed

  def MatrixCompn(self, MC, P, Iterations):

    d = {}
    for s in MC:
      if s.lower()=="optspace":
        M = MatrixCompletion(self.rank, Iterations)
        U, S, V = M.fit_transform(P.copy())
        Matrix = U@S@V.T
        d["optspace"] = Matrix
      
      elif s.lower() == "nm":
        Mask = self.mask(P.copy())
        Matrix = self.Matrix_Completion(P.copy(), self.rank, Mask, steps = Iterations)
        Matrix[P!=0] = P[P!=0]
        d["nm"] = Matrix

      elif s.lower()=="fancyimpute":
        with io.capture_output() as captured:
          Matrix = self.fancyimpute(P.copy(), Iterations)
        Matrix[P!=0] = P[P!=0]
        d["fancyimpute"] = Matrix

      else: 
        continue
    return d

  def Disagreements(self, P_true, MC, Known):

    d={}
    for key, P in MC.items():

      val = np.logical_xor((P>P.T), (P_true>P_true.T))
      val1 = np.logical_and(val, Known)
      d[key] = (val.sum(), val1.sum())
    return d

  def Upsets(self, P, sigma):
    
    n, = sigma.shape
    # print(sigma, P)
    count = 0
    for i in range(n):
      for j in range(0, i):
        if P[sigma[i]][sigma[j]] > P[sigma[j]][sigma[i]]:
          count+=1
    return count

  def Min_upsets(self, U, V, P, P_true):
    R = np.arctan2(V, U)
    index = np.argsort(R)
    n, = index.shape
    upset = n*n
    sigmaStar = index.copy()
    for i in range(n):
      rank_eval = self.Upsets(P, index)
      if rank_eval<upset:
        upset = rank_eval
        sigmaStar = index.copy()
      index = np.roll(index, 1)
    return self.Upsets(P_true, sigmaStar)

  def lowest_upsets(self, U,V,P):
    
    lowest=0
    for i in range(self.num_blocks):
      ind = np.arange(i*self.Block_size, (i+1)*self.Block_size, 1)
      lowest+=self.Min_upsets(U[np.ix_(ind)], V[np.ix_(ind)], P[np.ix_(ind, ind)], P[np.ix_(ind, ind)])
    return lowest

  def run(self, Seed, Iterations, K, Known_pairs, MC, Multiplier=1):

    rng = np.random.default_rng(seed = Seed)
    U, V, Score = self.Generate(rng)
    P_true = self.TrueP(U, V, Score)
    upsets = self.lowest_upsets(U, V, P_true)
    self.Random_shuffle(U, V, Score, P_true, rng)
    Known = self.KnownValues(Known_pairs, rng)
    P_sim = self.Simulation(rng, P_true, K)
    P_sim[Known==0]=0
    MC = self.MatrixCompn(MC, P_sim, Iterations)
    
    return P_true, MC, upsets

In [22]:
from tqdm import tqdm

In [23]:
def Run(n, blocksize, runs, k, known_fraction, iterations, mc):
  
  results= {}
  lowest = np.zeros(shape=(len(k), len(known_fraction)))
  allresults = np.empty([len(k), len(known_fraction)], dtype=object)
  for m in mc:
    results[m] = np.zeros(shape=(len(k), len(known_fraction)))

  SD = SyntheticData(n, blocksize)
  for iter in iterations:
    for i, trial in enumerate(k):
      for j, kf in tqdm(enumerate(known_fraction)):
        mean = testresults()
        mean.iter=iter
        mean.n=n
        mean.k=trial
        mean.kf=kf
        K = int(trial*4*blocksize*np.log(n)+0.5)
        Known_pairs = kf*(n//blocksize)*np.log(n)/n

        start_time = timeit.default_timer()
        for run in range(runs):
          P_true, P, min_upset = SD.run(run, iter, K, Known_pairs, mc)
          mean.min_upsets+=min_upset
          for algo, Matrix in P.items():
            PBR = PBR2R(Matrix, P_true)
            upset = PBR.BR2R_UPSETS(blocksize)
            mean.upsets[algo]+=upset

        elapsed = timeit.default_timer() - start_time
        mean.duration=elapsed/runs
        mean.min_upsets/=runs
        lowest[i,j]=mean.min_upsets
        for m in mc:
          mean.upsets[m]/=runs
          results[m][i,j] = mean.upsets[m]
        allresults[i,j]=mean
  return allresults, results, lowest

k= [5, 10]
known_fraction=[1,2,5,7.5,10]
iterations=[1000]
mc=["nm", "fancyimpute"]

In [ ]:
allresults, results, lowest = Run(600, 3, 10, k, known_fraction, iterations, mc)

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working


In [ ]:
np.save("/content/drive/MyDrive/Results/nmfi.npy", results)
np.save("/content/drive/MyDrive/Results/nmfioptimal.npy", lowest)